In [15]:
import aiomysql
import asyncio
from mo_dots import to_data, Data
import ujson
from pathlib import Path
import os
from pyprnt import prnt
from aiomysql import DictCursor

In [16]:
loop = asyncio.get_event_loop()
sql = await aiomysql.create_pool(host="127.0.0.1", port=3306,
                                 user="esurvey", password="",
                                 db="esurvey", loop=loop, autocommit=True)
async with sql.acquire() as conn:
    async with conn.cursor(DictCursor) as cur:
        icon_cache = Data()
        count = 0
        for file_path in Path("../api/collections-json/json").iterdir():
            lib_name = file_path.with_suffix('').name
            prnt(lib_name)
            with open(file_path) as lib_file:
                icolib = to_data(ujson.load(lib_file))
                for icon in icolib.icons:
                    #prnt(icon)
                    if icon in icon_cache:
                        icon_cache[icon].prefixes.append(lib_name)
                        #prnt(icon_cache[icon])
                    else:
                        icon_cache[icon]['prefixes'] = [lib_name]
                        #prnt(icon_cache[icon])
                    count += 1

                    await cur.execute("""
                                      INSERT INTO tags
                                      SET
                                          slug = %s,
                                          prefixes = %s
                                      """, (icon,
                                            ujson.dumps(icon_cache[icon]['prefixes']),)

print(count)

SyntaxError: invalid syntax (<ipython-input-16-df19bde82936>, line 32)